<a href="https://colab.research.google.com/github/Y4S1-Deep-Learning/Apple-Leaves-Desease-Detection---Supervised-Learning/blob/kaushalya/Mask_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Downloading Kaggle Data sets directory into Colab**

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir ~/.kaggle          #create a hidden folder in root directory

In [ ]:
!cp /content/......... /kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download hamishcrazeai/maize-in-field-dataset

**Installing Mask RCNN**

In [ ]:
!git clone https://github.com/akTwelve/Mask_RCNN.git

In [ ]:
import os
import sys
os.chdir('Mask_RCNN/samples')

In [ ]:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)